In [1]:
#分类训练模型  1. service_type=3 <=> current_service=99104722
#分类训练模型  2. service_type=1 <=> current_service={90063345: 0, 90109916: 1, 90155946: 2}
#分类训练模型  3. service_type=4 <=> current_service={89016252: 0, 89016253: 1, 89016259: 2, 89950166: 3, 89950167: 4, 89950168: 5, 99999825: 6, 99999826: 7, 99999827: 8, 99999828: 9, 99999830: 10}


# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# from sklearn.preprocessing import LabelEncoder
# from sklearn.metrics import accuracy_score
# from sklearn.feature_selection import VarianceThreshold
# import lightgbm as lgb
# import warnings
# from sklearn.preprocessing import OneHotEncoder
# import zipfile
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# warnings.filterwarnings('ignore')
# n_splits = 3
# seed = 2019
# path = 'E:/BDCI Match/'

# #文件路径
# traindataFile = path + 'train.csv'
# testdataFile = path+ 'test.csv'

# #文件加载
# train = pd.read_csv(traindataFile)
# test = pd.read_csv(testdataFile)

# #current_service为99104722的记录可以剔除，因为service_type=3 <=> current_service=99104722
# train1 = train.loc[train['service_type']==1]
# train1 = train1.reset_index()

# train2 = train.loc[train['service_type']==4]
# train2 = train2.reset_index()

# test1 = test.loc[test['service_type']==1]
# test1 = test1.reset_index()
# test2 = test.loc[test['service_type']==4]
# test2 = test2.reset_index()
# test3 = test.loc[test['service_type']==3]
# test3 = test3.reset_index()
# test_id3 = test3['user_id']

# # 对标签编码 映射关系
# label2current_service1 = dict(zip(range(0,len(set(train1['current_service']))),sorted(list(set(train1['current_service'])))))
# current_service2label1 = dict(zip(sorted(list(set(train1['current_service']))),range(0,len(set(train1['current_service'])))))

# label2current_service2 = dict(zip(range(0,len(set(train2['current_service']))),sorted(list(set(train2['current_service'])))))
# current_service2label2 = dict(zip(sorted(list(set(train2['current_service']))),range(0,len(set(train2['current_service'])))))

# # 原始数据的标签映射
# train1['current_service'] = train1['current_service'].map(current_service2label1)
# train2['current_service'] = train2['current_service'].map(current_service2label2)

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import VarianceThreshold
import lightgbm as lgb
import warnings
from sklearn.preprocessing import OneHotEncoder
import zipfile
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings('ignore')
n_splits = 3
seed = 2019
path = 'E:/BDCI Match/'

#文件路径
traindataFile = path + 'train.csv'
testdataFile = path+ 'republish_test.csv'
train_supdataFile = path + 'train2/test.csv'
train_supdataFileLabel = path + 'train2/standard.csv'

#补充训练集处理
train_sup = pd.read_csv(train_supdataFile)
train_supLabel = pd.read_csv(train_supdataFileLabel)
train_sup = pd.merge(train_sup,train_supLabel,on = 'user_id')

#文件加载
train = pd.read_csv(traindataFile)
test = pd.read_csv(testdataFile)

#合并训练集
train = train.append(train_sup,ignore_index = True)
train

#current_service为99104722的记录可以剔除，因为service_type=3 <=> current_service=99104722
train1 = train.loc[train['service_type']==1]
train1 = train1.reset_index()

train2 = train.loc[train['service_type']==4]
train2 = train2.reset_index()

test1 = test.loc[test['service_type']==1]
test1 = test1.reset_index()
test2 = test.loc[test['service_type']==4]
test2 = test2.reset_index()
test3 = test.loc[test['service_type']==3]
test3 = test3.reset_index()
test_id3 = test3['user_id']

# 对标签编码 映射关系
label2current_service1 = dict(zip(range(0,len(set(train1['current_service']))),sorted(list(set(train1['current_service'])))))
current_service2label1 = dict(zip(sorted(list(set(train1['current_service']))),range(0,len(set(train1['current_service'])))))

label2current_service2 = dict(zip(range(0,len(set(train2['current_service']))),sorted(list(set(train2['current_service'])))))
current_service2label2 = dict(zip(sorted(list(set(train2['current_service']))),range(0,len(set(train2['current_service'])))))

# 原始数据的标签映射
train1['current_service'] = train1['current_service'].map(current_service2label1)
train2['current_service'] = train2['current_service'].map(current_service2label2)

print('train1 data shape',train1.shape)
print('train1 data of user_id shape',len(set(train1['user_id'])))
print('train1 data of current_service shape',current_service2label1)

print('test1 data shape',test1.shape)
print('test1 data of user_id shape',len(set(test1['user_id'])))

print('train2 data shape',train2.shape)
print('train2 data of user_id shape',len(set(train2['user_id'])))
print('train2 data of current_service shape',current_service2label2)

print('test2 data shape',test2.shape)
print('test2 data of user_id shape',len(set(test2['user_id'])))

print('test3 data shape',test3.shape)
print('test3 data of user_id shape',len(set(test3['user_id'])))


# t1 = train1.loc[(train1['current_service']!=0)&(train1['current_service']!=1)&(train1['current_service']!=2)]
# print(t1)
# exit()

# 构造原始数据
index1 = train1.pop('index')
y1 = train1.pop('current_service')
train_id1 = train1.pop('user_id')
train1.pop('service_type')
X1 = train1
train_col = train1.columns

X1_test = test1[train_col]
test_id1 = test1['user_id']

# 构造原始数据
index2 = train2.pop('index')
y2 = train2.pop('current_service')
train_id2 = train2.pop('user_id')
train2.pop('service_type')
X2 = train2
train_col = train2.columns

X2_test = test2[train_col]
test_id2 = test2['user_id']

# 数据有问题数据
for i in train_col:
    X1[i] = X1[i].replace("\\N",-1)
    X2[i] = X2[i].replace("\\N",-1)
    X1_test[i] = X1_test[i].replace("\\N",-1)
    X2_test[i] = X2_test[i].replace("\\N",-1)

    
X1,y1,X1_test = X1.values,y1,X1_test.values
X2,y2,X2_test = X2.values,y2,X2_test.values


# 采取k折模型方案
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import numpy as np

# lgb 参数
params1={
    "learning_rate":0.02,
    "lambda_l1":0.1,
    "lambda_l2":0.2,
    "max_depth":-1,
    "objective":"multiclass",
    "num_class":3,
    "silent":True,
}

params2={
    "learning_rate":0.02,
    "lambda_l1":0.1,
    "lambda_l2":0.2,
    "max_depth":-1,
    "objective":"multiclass",
    "num_class":11,
    "silent":True,
}

# 自定义F1评价函数
def f1_score_vali3(preds, data_vali):
#     print(preds.shape)
#     print(preds)
    labels = data_vali.get_label()
    preds = np.argmax(preds.reshape(3, -1), axis=0)
    score_vali = f1_score(y_true=labels, y_pred=preds, average='weighted')
    return 'f1_score', score_vali, True

# 自定义F1评价函数
def f1_score_vali11(preds, data_vali):
#     print(preds.shape)
#     print(preds)
    labels = data_vali.get_label()
    preds = np.argmax(preds.reshape(11, -1), axis=0)
    score_vali = f1_score(y_true=labels, y_pred=preds, average='weighted')
    return 'f1_score', score_vali, True

#k折训练模型
def trainKfold(X, y, X_test, params):
    xx_score = []
    cv_pred = []
   
    skf = StratifiedKFold(n_splits=n_splits,random_state=seed,shuffle=True)
    for index,(train_index,test_index) in enumerate(skf.split(X,y)):
        print(index)

        X_train,X_valid,y_train,y_valid = X[train_index],X[test_index],y[train_index],y[test_index]
        
        train_data = lgb.Dataset(X_train, label=y_train)
        validation_data = lgb.Dataset(X_valid, label=y_valid)
        if params['num_class'] ==3:
            clf=lgb.train(params,train_data,num_boost_round=10000,valid_sets=[validation_data],early_stopping_rounds=100,feval=f1_score_vali3,verbose_eval=20)
        else:
            clf=lgb.train(params,train_data,num_boost_round=10000,valid_sets=[validation_data],early_stopping_rounds=100,feval=f1_score_vali11,verbose_eval=20)

        xx_pred = clf.predict(X_valid,num_iteration=clf.best_iteration)

        xx_pred = [np.argmax(x) for x in xx_pred]

        xx_score.append(f1_score(y_valid,xx_pred,average='weighted'))

        y_test = clf.predict(X_test,num_iteration=clf.best_iteration)

        y_test = [np.argmax(x) for x in y_test]

        if index == 0:
            cv_pred = np.array(y_test).reshape(-1, 1)
        else:
            cv_pred = np.hstack((cv_pred, np.array(y_test).reshape(-1, 1)))

    # 投票
    submit = []
    for line in cv_pred:
        submit.append(np.argmax(np.bincount(line)))
    return submit

submit1 = trainKfold(X1, y1, X1_test, params1)
submit2 = trainKfold(X2, y2, X2_test, params2)
submit3 = [99104722]*len(list(test_id3.unique()))


temp1 = pd.DataFrame()
temp2 = pd.DataFrame()
temp1['submit1'] = submit1
temp1['submit1'] = temp1['submit1'].map(label2current_service1)
temp2['submit2'] = submit2
temp2['submit2'] = temp2['submit2'].map(label2current_service2)

# 保存结果
df_test = pd.DataFrame()
df_test['id'] = list(test_id1.unique())+list(test_id2.unique())+list(test_id3.unique())
df_test['predict'] = list(temp1['submit1'])+list(temp2['submit2'])+list(submit3)
# df_test['predict'] = df_test['predict'].map(label2current_service)

df_test.to_csv(path+ './result/sample_submission.csv',index=False)


train1 data shape (347352, 28)
train1 data of user_id shape 347352
train1 data of current_service shape {90063345: 0, 90109916: 1, 90155946: 2}
test1 data shape (132818, 27)
test1 data of user_id shape 132818
train2 data shape (475870, 28)
train2 data of user_id shape 475870
train2 data of current_service shape {89016252: 0, 89016253: 1, 89016259: 2, 89950166: 3, 89950167: 4, 89950168: 5, 99999825: 6, 99999826: 7, 99999827: 8, 99999828: 9, 99999830: 10}
test2 data shape (67163, 27)
test2 data of user_id shape 67163
test3 data shape (19, 27)
test3 data of user_id shape 19
0
Training until validation scores don't improve for 100 rounds.
[20]	valid_0's multi_logloss: 0.674371	valid_0's f1_score: 0.987215
[40]	valid_0's multi_logloss: 0.437017	valid_0's f1_score: 0.988582
[60]	valid_0's multi_logloss: 0.292754	valid_0's f1_score: 0.989774
[80]	valid_0's multi_logloss: 0.201449	valid_0's f1_score: 0.990416
[100]	valid_0's multi_logloss: 0.142073	valid_0's f1_score: 0.990791
[120]	valid_0's 

[180]	valid_0's multi_logloss: 0.0451656	valid_0's f1_score: 0.992735
[200]	valid_0's multi_logloss: 0.0366152	valid_0's f1_score: 0.993079
[220]	valid_0's multi_logloss: 0.0306374	valid_0's f1_score: 0.993338
[240]	valid_0's multi_logloss: 0.0263061	valid_0's f1_score: 0.99358
[260]	valid_0's multi_logloss: 0.0231702	valid_0's f1_score: 0.993751
[280]	valid_0's multi_logloss: 0.0209112	valid_0's f1_score: 0.993925
[300]	valid_0's multi_logloss: 0.0192284	valid_0's f1_score: 0.994203
[320]	valid_0's multi_logloss: 0.0179683	valid_0's f1_score: 0.994443
[340]	valid_0's multi_logloss: 0.0169861	valid_0's f1_score: 0.994607
[360]	valid_0's multi_logloss: 0.0162232	valid_0's f1_score: 0.99472
[380]	valid_0's multi_logloss: 0.015649	valid_0's f1_score: 0.994824
[400]	valid_0's multi_logloss: 0.0151586	valid_0's f1_score: 0.994953
[420]	valid_0's multi_logloss: 0.0147461	valid_0's f1_score: 0.995039
[440]	valid_0's multi_logloss: 0.0143999	valid_0's f1_score: 0.995177
[460]	valid_0's multi_l

[1260]	valid_0's multi_logloss: 0.352298	valid_0's f1_score: 0.871098
[1280]	valid_0's multi_logloss: 0.351733	valid_0's f1_score: 0.871232
[1300]	valid_0's multi_logloss: 0.351159	valid_0's f1_score: 0.871466
[1320]	valid_0's multi_logloss: 0.350616	valid_0's f1_score: 0.871694
[1340]	valid_0's multi_logloss: 0.350092	valid_0's f1_score: 0.871863
[1360]	valid_0's multi_logloss: 0.349526	valid_0's f1_score: 0.87207
[1380]	valid_0's multi_logloss: 0.348977	valid_0's f1_score: 0.872136
[1400]	valid_0's multi_logloss: 0.348453	valid_0's f1_score: 0.872458
[1420]	valid_0's multi_logloss: 0.34797	valid_0's f1_score: 0.872553
[1440]	valid_0's multi_logloss: 0.347485	valid_0's f1_score: 0.872828
[1460]	valid_0's multi_logloss: 0.346965	valid_0's f1_score: 0.873019
[1480]	valid_0's multi_logloss: 0.346472	valid_0's f1_score: 0.873165
[1500]	valid_0's multi_logloss: 0.346003	valid_0's f1_score: 0.87328
[1520]	valid_0's multi_logloss: 0.345545	valid_0's f1_score: 0.873486
[1540]	valid_0's multi_

[3620]	valid_0's multi_logloss: 0.317396	valid_0's f1_score: 0.884447
[3640]	valid_0's multi_logloss: 0.317234	valid_0's f1_score: 0.884451
[3660]	valid_0's multi_logloss: 0.317075	valid_0's f1_score: 0.884536
[3680]	valid_0's multi_logloss: 0.316914	valid_0's f1_score: 0.884638
[3700]	valid_0's multi_logloss: 0.31676	valid_0's f1_score: 0.8847
[3720]	valid_0's multi_logloss: 0.316605	valid_0's f1_score: 0.884849
[3740]	valid_0's multi_logloss: 0.316463	valid_0's f1_score: 0.884933
[3760]	valid_0's multi_logloss: 0.316302	valid_0's f1_score: 0.884999
[3780]	valid_0's multi_logloss: 0.31617	valid_0's f1_score: 0.885007
[3800]	valid_0's multi_logloss: 0.316019	valid_0's f1_score: 0.885072
[3820]	valid_0's multi_logloss: 0.315876	valid_0's f1_score: 0.885089
[3840]	valid_0's multi_logloss: 0.315725	valid_0's f1_score: 0.885172
[3860]	valid_0's multi_logloss: 0.315583	valid_0's f1_score: 0.885263
[3880]	valid_0's multi_logloss: 0.315431	valid_0's f1_score: 0.885261
[3900]	valid_0's multi_l

[5980]	valid_0's multi_logloss: 0.304582	valid_0's f1_score: 0.890512
[6000]	valid_0's multi_logloss: 0.304506	valid_0's f1_score: 0.890506
[6020]	valid_0's multi_logloss: 0.304435	valid_0's f1_score: 0.890509
[6040]	valid_0's multi_logloss: 0.304373	valid_0's f1_score: 0.890573
[6060]	valid_0's multi_logloss: 0.304305	valid_0's f1_score: 0.890596
[6080]	valid_0's multi_logloss: 0.304239	valid_0's f1_score: 0.890588
[6100]	valid_0's multi_logloss: 0.304168	valid_0's f1_score: 0.890712
[6120]	valid_0's multi_logloss: 0.304091	valid_0's f1_score: 0.890769
[6140]	valid_0's multi_logloss: 0.304021	valid_0's f1_score: 0.890805
[6160]	valid_0's multi_logloss: 0.303962	valid_0's f1_score: 0.890844
[6180]	valid_0's multi_logloss: 0.30391	valid_0's f1_score: 0.890834
[6200]	valid_0's multi_logloss: 0.303834	valid_0's f1_score: 0.890849
[6220]	valid_0's multi_logloss: 0.303776	valid_0's f1_score: 0.890899
[6240]	valid_0's multi_logloss: 0.303727	valid_0's f1_score: 0.890916
[6260]	valid_0's mult

...

[3220]	valid_0's multi_logloss: 0.318383	valid_0's f1_score: 0.883991
[3240]	valid_0's multi_logloss: 0.318204	valid_0's f1_score: 0.884073
[3260]	valid_0's multi_logloss: 0.318023	valid_0's f1_score: 0.884177
[3280]	valid_0's multi_logloss: 0.317837	valid_0's f1_score: 0.88425
[3300]	valid_0's multi_logloss: 0.317673	valid_0's f1_score: 0.884413
[3320]	valid_0's multi_logloss: 0.317485	valid_0's f1_score: 0.884376
[3340]	valid_0's multi_logloss: 0.317306	valid_0's f1_score: 0.884463
[3360]	valid_0's multi_logloss: 0.317122	valid_0's f1_score: 0.884592
[3380]	valid_0's multi_logloss: 0.316929	valid_0's f1_score: 0.884687
[3400]	valid_0's multi_logloss: 0.316754	valid_0's f1_score: 0.884689
[3420]	valid_0's multi_logloss: 0.316567	valid_0's f1_score: 0.884813
[3440]	valid_0's multi_logloss: 0.316378	valid_0's f1_score: 0.885003
[3460]	valid_0's multi_logloss: 0.316192	valid_0's f1_score: 0.885031
[3480]	valid_0's multi_logloss: 0.316014	valid_0's f1_score: 0.885029
[3500]	valid_0's mult

[5580]	valid_0's multi_logloss: 0.303033	valid_0's f1_score: 0.89075
[5600]	valid_0's multi_logloss: 0.302935	valid_0's f1_score: 0.890793
[5620]	valid_0's multi_logloss: 0.302841	valid_0's f1_score: 0.890861
[5640]	valid_0's multi_logloss: 0.302748	valid_0's f1_score: 0.890774
[5660]	valid_0's multi_logloss: 0.302666	valid_0's f1_score: 0.890856
[5680]	valid_0's multi_logloss: 0.302587	valid_0's f1_score: 0.890786
[5700]	valid_0's multi_logloss: 0.302504	valid_0's f1_score: 0.890847
[5720]	valid_0's multi_logloss: 0.302424	valid_0's f1_score: 0.890859
[5740]	valid_0's multi_logloss: 0.302334	valid_0's f1_score: 0.890932
[5760]	valid_0's multi_logloss: 0.302253	valid_0's f1_score: 0.890923
[5780]	valid_0's multi_logloss: 0.302168	valid_0's f1_score: 0.890975
[5800]	valid_0's multi_logloss: 0.302093	valid_0's f1_score: 0.891108
[5820]	valid_0's multi_logloss: 0.30201	valid_0's f1_score: 0.89112
[5840]	valid_0's multi_logloss: 0.301928	valid_0's f1_score: 0.891216
[5860]	valid_0's multi_

[1180]	valid_0's multi_logloss: 0.356822	valid_0's f1_score: 0.869906
[1200]	valid_0's multi_logloss: 0.356199	valid_0's f1_score: 0.87001
[1220]	valid_0's multi_logloss: 0.355567	valid_0's f1_score: 0.870092
[1240]	valid_0's multi_logloss: 0.354941	valid_0's f1_score: 0.870417
[1260]	valid_0's multi_logloss: 0.354308	valid_0's f1_score: 0.870596
[1280]	valid_0's multi_logloss: 0.353775	valid_0's f1_score: 0.870777
[1300]	valid_0's multi_logloss: 0.353195	valid_0's f1_score: 0.870981
[1320]	valid_0's multi_logloss: 0.352616	valid_0's f1_score: 0.871135
[1340]	valid_0's multi_logloss: 0.352005	valid_0's f1_score: 0.87151
[1360]	valid_0's multi_logloss: 0.351434	valid_0's f1_score: 0.871658
[1380]	valid_0's multi_logloss: 0.350914	valid_0's f1_score: 0.871797
[1400]	valid_0's multi_logloss: 0.350368	valid_0's f1_score: 0.872025
[1420]	valid_0's multi_logloss: 0.349854	valid_0's f1_score: 0.872193
[1440]	valid_0's multi_logloss: 0.349328	valid_0's f1_score: 0.872399
[1460]	valid_0's multi

[3540]	valid_0's multi_logloss: 0.319451	valid_0's f1_score: 0.884285
[3560]	valid_0's multi_logloss: 0.319283	valid_0's f1_score: 0.8843
[3580]	valid_0's multi_logloss: 0.319121	valid_0's f1_score: 0.884352
[3600]	valid_0's multi_logloss: 0.318969	valid_0's f1_score: 0.884387
[3620]	valid_0's multi_logloss: 0.318816	valid_0's f1_score: 0.88446
[3640]	valid_0's multi_logloss: 0.31864	valid_0's f1_score: 0.884557
[3660]	valid_0's multi_logloss: 0.31846	valid_0's f1_score: 0.884557
[3680]	valid_0's multi_logloss: 0.318306	valid_0's f1_score: 0.884554
[3700]	valid_0's multi_logloss: 0.318143	valid_0's f1_score: 0.884628
[3720]	valid_0's multi_logloss: 0.317982	valid_0's f1_score: 0.88468
[3740]	valid_0's multi_logloss: 0.317836	valid_0's f1_score: 0.884665
[3760]	valid_0's multi_logloss: 0.317677	valid_0's f1_score: 0.884771
[3780]	valid_0's multi_logloss: 0.317537	valid_0's f1_score: 0.884918
[3800]	valid_0's multi_logloss: 0.317409	valid_0's f1_score: 0.884896
[3820]	valid_0's multi_log

[5900]	valid_0's multi_logloss: 0.306043	valid_0's f1_score: 0.890416
[5920]	valid_0's multi_logloss: 0.305984	valid_0's f1_score: 0.890443
[5940]	valid_0's multi_logloss: 0.305907	valid_0's f1_score: 0.890584
[5960]	valid_0's multi_logloss: 0.305834	valid_0's f1_score: 0.890551
[5980]	valid_0's multi_logloss: 0.305765	valid_0's f1_score: 0.890509
[6000]	valid_0's multi_logloss: 0.305686	valid_0's f1_score: 0.890496
[6020]	valid_0's multi_logloss: 0.305617	valid_0's f1_score: 0.890523
[6040]	valid_0's multi_logloss: 0.305548	valid_0's f1_score: 0.890542
Early stopping, best iteration is:
[5942]	valid_0's multi_logloss: 0.3059	valid_0's f1_score: 0.890604


In [5]:
#分类训练模型  1. service_type=3 <=> current_service=99104722
#分类训练模型  2. service_type=1 <=> current_service={90063345: 0, 90109916: 1, 90155946: 2}
#分类训练模型  3. service_type=4 <=> current_service={89016252: 0, 89016253: 1, 89016259: 2, 89950166: 3, 89950167: 4, 89950168: 5, 99999825: 6, 99999826: 7, 99999827: 8, 99999828: 9, 99999830: 10}


# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# from sklearn.preprocessing import LabelEncoder
# from sklearn.metrics import accuracy_score
# from sklearn.feature_selection import VarianceThreshold
# import lightgbm as lgb
# import warnings
# from sklearn.preprocessing import OneHotEncoder
# import zipfile
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# warnings.filterwarnings('ignore')
# n_splits = 3
# seed = 2019
# path = 'E:/BDCI Match/'

# #文件路径
# traindataFile = path + 'train.csv'
# testdataFile = path+ 'test.csv'

# #文件加载
# train = pd.read_csv(traindataFile)
# test = pd.read_csv(testdataFile)

# #current_service为99104722的记录可以剔除，因为service_type=3 <=> current_service=99104722
# train1 = train.loc[train['service_type']==1]
# train1 = train1.reset_index()

# train2 = train.loc[train['service_type']==4]
# train2 = train2.reset_index()

# test1 = test.loc[test['service_type']==1]
# test1 = test1.reset_index()
# test2 = test.loc[test['service_type']==4]
# test2 = test2.reset_index()
# test3 = test.loc[test['service_type']==3]
# test3 = test3.reset_index()
# test_id3 = test3['user_id']

# # 对标签编码 映射关系
# label2current_service1 = dict(zip(range(0,len(set(train1['current_service']))),sorted(list(set(train1['current_service'])))))
# current_service2label1 = dict(zip(sorted(list(set(train1['current_service']))),range(0,len(set(train1['current_service'])))))

# label2current_service2 = dict(zip(range(0,len(set(train2['current_service']))),sorted(list(set(train2['current_service'])))))
# current_service2label2 = dict(zip(sorted(list(set(train2['current_service']))),range(0,len(set(train2['current_service'])))))

# # 原始数据的标签映射
# train1['current_service'] = train1['current_service'].map(current_service2label1)
# train2['current_service'] = train2['current_service'].map(current_service2label2)

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import VarianceThreshold
import lightgbm as lgb
import warnings
from sklearn.preprocessing import OneHotEncoder
import zipfile
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings('ignore')
n_splits = 3
seed = 2019
path = 'E:/BDCI Match/'

#文件路径
traindataFile = path + 'train.csv'
testdataFile = path+ 'republish_test.csv'
train_supdataFile = path + 'train2/test.csv'
train_supdataFileLabel = path + 'train2/standard.csv'

#补充训练集处理
train_sup = pd.read_csv(train_supdataFile)
train_supLabel = pd.read_csv(train_supdataFileLabel)
train_sup = pd.merge(train_sup,train_supLabel,on = 'user_id')

#文件加载
train = pd.read_csv(traindataFile)
test = pd.read_csv(testdataFile)

#合并训练集
train = train.append(train_sup,ignore_index = True)
temp = train.loc[train['current_service']==99104722]
temp.describe()



,1_total_fee,4_total_fee,complaint_level,contract_time,contract_type,current_service,former_complaint_fee,former_complaint_num,is_mix_service,is_promise_low_consume,...,local_trafffic_month,many_over_bill,month_traffic,net_service,online_time,pay_num,pay_times,service1_caller_time,service2_caller_time,service_type
count,51995.000000,51995.000000,51995.000000,51995.0,51995.0,51995.0,51995.000000,51995.000000,51995.0,51995.000000,...,51995.000000,51995.000000,51995.0,51995.000000,51995.000000,51995.000000,51995.000000,51995.000000,51995.000000,51995.0
mean,46.561293,50.725862,0.115742,0.0,0.0,99104722.0,334.473142,0.139706,0.0,0.057294,...,5557.988121,0.112588,0.0,3.926435,26.059390,50.273058,1.618233,0.291602,0.752176,3.0
std,54.869009,65.332889,0.382611,0.0,0.0,0.0,2504.063288,0.512887,0.0,0.232406,...,8218.803767,0.316091,0.0,0.333406,20.522471,63.882419,1.329608,0.980653,1.046254,0.0
min,0.000000,-2.160000,0.000000,0.0,0.0,99104722.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,2.000000,4.000000,0.100000,1.000000,0.000000,0.000000,3.0
25%,13.000000,11.000000,0.000000,0.0,0.0,99104722.0,0.000000,0.000000,0.0,0.000000,...,934.250977,0.000000,0.0,4.000000,8.000000,10.000000,1.000000,0.000000,0.090833,3.0
50%,33.550000,33.000000,0.000000,0.0,0.0,99104722.0,0.000000,0.000000,0.0,0.000000,...,2661.670898,0.000000,0.0,4.000000,20.000000,30.000000,1.000000,0.000000,0.476389,3.0
75%,57.305000,60.000000,0.000000,0.0,0.0,99104722.0,0.000000,0.000000,0.0,0.000000,...,7103.660644,0.000000,0.0,4.000000,44.000000,60.000000,2.000000,0.000000,1.001389,3.0
max,1142.140000,998.040000,3.000000,0.0,0.0,99104722.0,89545.000000,23.000000,0.0,1.000000,...,107563.449200,1.000000,0.0,9.000000,224.000000,1100.000000,42.000000,52.854167,34.239444,3.0
